- Use repo provided from Anthropic: {https://github.com/anthropics/anthropic-cookbook/tree/main/skills/contextual-embeddings}

In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### Preprocess the "documents" in openbook.txt:

In [4]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

with open("data\Main\openbook.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
documents = [line.strip() for line in lines]

embedder = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

### Test out queries:

In [7]:
# Query embedding
query = "Tell me about dogs."
query_embedding = embedder.encode([query])

# Search FAISS index
D, I = index.search(np.array(query_embedding), k=1)
print("Best match:", documents[I[0][0]])

Best match: "animals live and feed near their habitats"
